## Create custom Views (grid channels)

To make custom channels, you have to work with the View class. This class helps you make channels that you design yourself.

The basic idea is to create a boolean matrix with dimensions: `n_channels` (how many channels you want) and `n_atoms_complex` (how many atoms are in your protein-ligand complex). In this grid, each row is a different channel (like one for carbon atoms, one for hydrogen-bond donors, and so on). Each column represents an atom in your complex. A spot in the grid tells you if that atom is in the channel or not.

Also, note that the atoms follow the order they appear in the PDB file. The atoms from the protein are listed first, followed by those from the ligand.

The View class has five main methods you need to fill in:

* `get_num_channels()`: Tells how many channels there are.
* `get_channels_names()`: Gives the names of your channels.
* `get_molecular_complex_channels(molecular_complex: MolecularComplex)`: Sets up channels for the whole molecular complex.
* `get_protein_channels(molecular_complex: MolecularComplex)`: Sets up channels just for the protein.
* `get_ligand_channels(molecular_complex: MolecularComplex)`: Sets up channels just for the ligand.

These last three methods make it easier to handle the channels for each part of your complex separately.

In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np

from docktgrid.view import View
from docktgrid.molecule import MolecularComplex
from docktgrid.molparser import MolecularParser

Here's how we can use our setup for making custom channels. As an example, consider a grid construction
where each channel selectively includes protein atoms based on their distance from the 
ligand's center. More specifically, for channel $i$ (with $i$ ranging from 0 to $n$), 
it encompasses protein atoms at a distance $d_a$ satisfying the condition 
$x_i \leq d_a < y_i$. Here, $x_i$ and $y_i$ serve as the defining parameters for 
distance boundaries.

We define these parameters as:

$$
(x_0, y_0) = (0, 5) \\
(x_1, y_1) = (5, 10) \\
(x_2, y_2) = (10, 15) \\
(x_3, y_3) = (15, 20)
$$

This would give us something like the figure below, where colors represent the different channels with increasing distance boundaries (gray, light pink, dark pink and orange):

> ![](figures/custom_view.png)


For the ligand let's just use a single channel that includes every atom. 
So the total number of channels for this representation is 4 + 1 = 5.

In [3]:
class CustomView(View):
    """Interface for defining voxel channels representations.

    See View class docs for a more complete documentation.
    """

    def get_num_channels(self):
        """Return number of channels defined for the view."""
        n_channels_complex = 0  # we can skip these since we are not using them
        n_channels_protein = 4
        n_channels_ligand = 1
        return sum((n_channels_complex, n_channels_protein, n_channels_ligand))

    def get_channels_names(self):
        """Return names of channels defined for the view."""
        pass  # this is only useful for visualizations, exporting etc.

    def get_molecular_complex_channels(self, molecular_complex: MolecularComplex):
        """Set of channels considering all atoms of the protein-ligand complex together."""
        return None  # return an empty tensor since we are not using this

    def get_ligand_channels(self, molecular_complex: MolecularComplex):
        """Set of channels considering ligand atoms only."""
        ligand = torch.zeros((1, molecular_complex.n_atoms), dtype=torch.bool)

        # only ligand atoms are true, since this function does not consider
        # protein atoms (which are set to zero)

        # important: n_atoms = protein_atoms followed by ligand_atoms (in this order)
        ligand[0][-molecular_complex.n_atoms_ligand :] = True
        return ligand

    def get_protein_channels(self, molecular_complex: MolecularComplex):
        """Set of channels considering protein atoms only."""
        # this is a biopandas object of the protein file, you can see the docs here:
        # https://biopandas.github.io/biopandas/ and do whatever you want with it
        protein = molecular_complex.protein_data.molecule_object
        ligand_center = molecular_complex.ligand_center
        n_atoms_protein = molecular_complex.n_atoms_protein

        distances = [(0, 5), (5, 10), (10, 15), (15, 20)]
        channels = torch.zeros((len(distances), molecular_complex.n_atoms), dtype=torch.bool)
        d = protein.distance(ligand_center)
        
        for i, (x, y) in enumerate(distances):
            dst = ((d >= x) & (d < y)).to_numpy()
            channels[i, :n_atoms_protein] = torch.tensor(dst, dtype=torch.bool)
        
        return channels

In [4]:
protein_path = "../tests/data/{}_protein.pdb"
ligand_path = "../tests/data/{}_ligand.pdb"
id_ = "6rnt"

mol = MolecularComplex(protein_path.format(id_), ligand_path.format(id_), MolecularParser())

In [6]:
custom_view = CustomView()
custom_view(mol).shape

torch.Size([5, 988])